[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/182tLTa3DzVHb01A-Lsy9PAO49pI2m5G4?usp=sharing)

# Temperature Response Functions

Here we use the temperature **Trapezoid temperature function** with different cardinal temperature combinations with and without VPD stress function for the period from heading to maturity (grain filling period). 

Authors: 
-    **Azam Lashkari** (CIMMYT-China, azam.lashkari@jic.ac.uk)
-    **Urs Christoph Schulthess** (CIMMYT-China, U.Schulthess@cgiar.org)
-    **Ernesto Giron Echeverry** (Independent Researcher, e.giron.e@gmail.com)

Last updated: October 01, 2023


## Querying combinations with precision using DuckDB


In [13]:
# to install: pip install duckdb
import duckdb

# some DuckDB setup 
con = duckdb.connect()
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

## First query

Now suppose we want to figure out how many rows are in our data set. We can do that using the following code snippets.

### Display the number of combinations in TPF simulations

In [13]:
# Display the number of combinations in TPF simulations
print(duckdb.query('''
SELECT COUNT(*)
FROM '/Users/ernestogiron/Desktop/TemperatureFunctions/results/WETF/WETF_SFvpd/combinations_Yield_WETF_SFvpd.parquet' 
''').fetchall())

[(15680000,)]


### Filter dataset

We can filter the big files easily as follows:

In [17]:
%%time
print(duckdb.query('''
SELECT COUNT(*)
FROM '/Users/ernestogiron/Desktop/TemperatureFunctions/results/WETF/WETF_SFvpd/combinations_Yield_WETF_SFvpd.parquet' 
WHERE RUE = 3.0 AND TminFactor= 0.25 AND Topt BETWEEN 16.0 AND 18
''').fetchall())

[(2956800,)]
CPU times: user 349 ms, sys: 7.35 ms, total: 356 ms
Wall time: 59 ms


### Display result in a table or pandas dataframe

In [16]:
%%time
con.execute("SELECT * FROM '/Users/ernestogiron/Desktop/TemperatureFunctions/results/WETF/WETF_SFvpd/combinations_Yield_WETF_SFvpd.parquet' \
WHERE RUE = 3.0 AND TminFactor= 0.25 AND Topt BETWEEN 16.0 AND 18 LIMIT 5").df()


CPU times: user 35.6 ms, sys: 7.11 ms, total: 42.7 ms
Wall time: 22.1 ms


,UID,country,location,loc_code,cycle,ObsYield,RUE,Tmin,Topt,Tmax,TminFactor,Lvpd,Uvpd,SFvpd_Lthres,SFvpd_Uthres,SimYield
0,1,Nepal,Bhairahawa,BHR,2019,2.97,3.0,5.0,17.0,38.0,0.25,0.5,1.0,0.2,1.0,0.58
1,2,Pakistan,Faisalabad,FAS,2014,4.88,3.0,5.0,17.0,38.0,0.25,0.5,1.0,0.2,1.0,1.00
2,3,Pakistan,Faisalabad,FAS,2015,3.59,3.0,5.0,17.0,38.0,0.25,0.5,1.0,0.2,1.0,1.22
3,4,Pakistan,Faisalabad,FAS,2016,2.86,3.0,5.0,17.0,38.0,0.25,0.5,1.0,0.2,1.0,1.03
4,5,Pakistan,Faisalabad,FAS,2017,3.93,3.0,5.0,17.0,38.0,0.25,0.5,1.0,0.2,1.0,0.85


### Join combinations and metrics results in one table

In [18]:
%%time
con.execute("SELECT * \
FROM '/Users/ernestogiron/Desktop/TemperatureFunctions/results/WETF/WETF_SFvpd/combinations_Yield_WETF_SFvpd.parquet', \
'/Users/ernestogiron/Desktop/TemperatureFunctions/results/WETF/WETF_SFvpd/metrics_Yield_WETF_SFvpd.parquet' \
WHERE UID = UID LIMIT 5").df()


CPU times: user 82 ms, sys: 41.4 ms, total: 123 ms
Wall time: 123 ms


,UID,country,location,loc_code,cycle,ObsYield,RUE,Tmin,Topt,Tmax,...,RMSRE,MAPE,pvalue,R2,EF,intercept,slope,Cb,CCC,Accuracy
0,1,Nepal,Bhairahawa,BHR,2019,2.97,3.0,5.0,17.0,38.0,...,86.331,85.462,0.003508,0.164,-5.772,0.0504,0.1357,0.09,0.04,14.54
1,2,Pakistan,Faisalabad,FAS,2014,4.88,3.0,5.0,17.0,38.0,...,86.331,85.462,0.003508,0.164,-5.772,0.0504,0.1357,0.09,0.04,14.54
2,3,Pakistan,Faisalabad,FAS,2015,3.59,3.0,5.0,17.0,38.0,...,86.331,85.462,0.003508,0.164,-5.772,0.0504,0.1357,0.09,0.04,14.54
3,4,Pakistan,Faisalabad,FAS,2016,2.86,3.0,5.0,17.0,38.0,...,86.331,85.462,0.003508,0.164,-5.772,0.0504,0.1357,0.09,0.04,14.54
4,5,Pakistan,Faisalabad,FAS,2017,3.93,3.0,5.0,17.0,38.0,...,86.331,85.462,0.003508,0.164,-5.772,0.0504,0.1357,0.09,0.04,14.54
